#**Índicador da Média de horas aula diária por escola - 2020**
**Link dos Dados de origem:**<br>
https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/media-de-horas-aula-diaria
<br>
Responsável: Gustavo de Andrade - gustavo.andrade@prefeitura.sp.gov.br

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime, date, timedelta

In [16]:
dados = pd.read_excel("HAD_ESCOLAS_2020.xlsx", skiprows = range(0, 8))

In [17]:
dados

,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,ED_INF_CAT_0,CRE_CAT_0,PRE_CAT_0,FUN_CAT_0,FUN_AI_CAT_0,FUN_AF_CAT_0,FUN_01_CAT_0,FUN_02_CAT_0,FUN_03_CAT_0,FUN_04_CAT_0,FUN_05_CAT_0,FUN_06_CAT_0,FUN_07_CAT_0,FUN_08_CAT_0,FUN_09_CAT_0,MED_CAT_0,MED_01_CAT_0,MED_02_CAT_0,MED_03_CAT_0,MED_04_CAT_0,MED_NS_CAT_0
0,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11022558.0,EIEEF HAP BITT TUPARI,Rural,Estadual,--,--,--,4,4,--,--,4,4,4,4,--,--,--,--,--,--,--,--,--,--
1,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11024372.0,EMEIEF ANA NERY,Urbana,Municipal,4,--,4,4,4,4,--,--,4,4,4,4,4,4,4,--,--,--,--,--,--
2,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11024666.0,EMEIEF BOA ESPERANCA,Rural,Municipal,4,--,4,4,4,4,--,--,4,4,4,4,4,4,4,--,--,--,--,--,--
3,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11024682.0,EEEF EURIDICE LOPES PEDROSO,Urbana,Estadual,--,--,--,4,4,4,--,--,--,--,4,4,4,4,--,3.9,3.9,3.8,3.8,--,--
4,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11024828.0,EMEIEF IZIDORO STEDILE,Urbana,Municipal,4,--,4,4,4,4,4,4,4,4,4,4,4,4,4,--,--,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151920,2020,Centro-Oeste,DF,5300108.0,Brasília,53085000.0,COBIAN - COL BIANGULO,Urbana,Privada,4.2,3.8,4.4,4.5,4.5,--,4.5,4.5,4.5,--,--,--,--,--,--,--,--,--,--,--,--
151921,2020,Centro-Oeste,DF,5300108.0,Brasília,53086007.0,CED VITORIA REGIA,Urbana,Privada,--,--,--,4.9,--,4.9,--,--,--,--,--,--,--,4.4,5.3,5.5,5.7,5.3,5.3,--,--
151922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151923,Fonte: Censo da Educação Básica 2020/INEP.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filtrando - 'CO_MUNICIPIO' = 3550308 -> São Paulo

In [18]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151925 entries, 0 to 151924
Data columns (total 30 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   NU_ANO_CENSO    151924 non-null  object 
 1   NO_REGIAO       151922 non-null  object 
 2   SG_UF           151922 non-null  object 
 3   CO_MUNICIPIO    151922 non-null  float64
 4   NO_MUNICIPIO    151922 non-null  object 
 5   CO_ENTIDADE     151922 non-null  float64
 6   NO_ENTIDADE     151922 non-null  object 
 7   NO_CATEGORIA    151922 non-null  object 
 8   NO_DEPENDENCIA  151922 non-null  object 
 9   ED_INF_CAT_0    151922 non-null  object 
 10  CRE_CAT_0       151922 non-null  object 
 11  PRE_CAT_0       151922 non-null  object 
 12  FUN_CAT_0       151922 non-null  object 
 13  FUN_AI_CAT_0    151922 non-null  object 
 14  FUN_AF_CAT_0    151922 non-null  object 
 15  FUN_01_CAT_0    151922 non-null  object 
 16  FUN_02_CAT_0    151922 non-null  object 
 17  FUN_03_CAT

In [19]:
indicador_media_horas_aula = dados.loc[lambda dados: dados["NO_MUNICIPIO"] == "São Paulo"]
indicador_media_horas_aula

,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,ED_INF_CAT_0,CRE_CAT_0,PRE_CAT_0,FUN_CAT_0,FUN_AI_CAT_0,FUN_AF_CAT_0,FUN_01_CAT_0,FUN_02_CAT_0,FUN_03_CAT_0,FUN_04_CAT_0,FUN_05_CAT_0,FUN_06_CAT_0,FUN_07_CAT_0,FUN_08_CAT_0,FUN_09_CAT_0,MED_CAT_0,MED_01_CAT_0,MED_02_CAT_0,MED_03_CAT_0,MED_04_CAT_0,MED_NS_CAT_0
109290,2020,Sudeste,SP,3550308.0,São Paulo,35000012.0,AYRES DE MOURA PROFESSOR,Urbana,Estadual,--,--,--,8.7,--,8.7,--,--,--,--,--,8.5,8.8,8.8,8.8,--,--,--,--,--,--
109291,2020,Sudeste,SP,3550308.0,São Paulo,35000024.0,GAVIAO PEIXOTO BRIGADEIRO,Urbana,Estadual,--,--,--,5.4,5.3,5.5,4.9,5.3,5.3,5.3,5.3,5.3,5.3,5.6,5.6,5,5.3,4.8,4.7,--,--
109292,2020,Sudeste,SP,3550308.0,São Paulo,35000036.0,JOAO SOLIMEO,Urbana,Estadual,--,--,--,5.5,--,5.5,--,--,--,--,--,5.4,5.6,5.6,5.6,4.6,5.5,4,4,--,--
109293,2020,Sudeste,SP,3550308.0,São Paulo,35000048.0,WALFREDO ARANTES CALDAS PROFESSOR,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,5,5.2,4.9,4.5,--,--
109294,2020,Sudeste,SP,3550308.0,São Paulo,35000059.0,SEBASTIAO DE OLIVEIRA GUSMAO PROFESSOR,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,5,5.6,4.6,4.5,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116237,2020,Sudeste,SP,3550308.0,São Paulo,35925949.0,DOM AGNELO CARDEAL ROSSI,Urbana,Estadual,--,--,--,5.3,5.3,--,5.3,5.3,5.3,5.3,5.3,--,--,--,--,--,--,--,--,--,--
116238,2020,Sudeste,SP,3550308.0,São Paulo,35925998.0,ADELAIDE ROSA FERNANDES MACHADO DE SOUZA PROFE...,Urbana,Estadual,--,--,--,9.5,--,9.5,--,--,--,--,--,9.5,9.5,9.5,9.5,9.5,9.5,9.5,9.5,--,--
116239,2020,Sudeste,SP,3550308.0,São Paulo,35926048.0,BARRO BRANCO II,Urbana,Estadual,--,--,--,5,4.5,5.6,4.5,4.5,4.5,4.5,4.5,5.6,5.6,5.6,5.6,4,4,4,4,--,--
116240,2020,Sudeste,SP,3550308.0,São Paulo,35926103.0,PROFESSORA ELIANA PASSUELO,Urbana,Estadual,--,--,--,4.5,4.5,--,4.5,4.5,4.5,4.5,4.5,--,--,--,--,--,--,--,--,--,--


In [20]:
len(indicador_media_horas_aula)

6952

#**Modelando a tabela**

Limpando colunas desnecessárias:

In [21]:
indicador_media_horas_aula.drop(columns =['NU_ANO_CENSO','NO_REGIAO','SG_UF','CO_MUNICIPIO','NO_MUNICIPIO'], inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
indicador_media_horas_aula.sample(2)

,id_escola,escola,categoria,tipo,ED_INF_CAT_0,CRE_CAT_0,PRE_CAT_0,FUN_CAT_0,FUN_AI_CAT_0,FUN_AF_CAT_0,FUN_01_CAT_0,FUN_02_CAT_0,FUN_03_CAT_0,FUN_04_CAT_0,FUN_05_CAT_0,FUN_06_CAT_0,FUN_07_CAT_0,FUN_08_CAT_0,FUN_09_CAT_0,MED_CAT_0,MED_01_CAT_0,MED_02_CAT_0,MED_03_CAT_0,MED_04_CAT_0,MED_NS_CAT_0
114736,35456731.0,OBJETIVO CENTRO INTERESCOLAR UNIDADE IPIRANGA,Urbana,Privada,4.7,4.7,4.7,5.1,4.7,5.3,4.7,4.7,4.7,4.7,4.7,5.3,5.3,5.3,5.3,5.7,5.7,5.7,5.7,--,--
115517,35580004.0,CR P CONV MUNDINHO DO MATHEUS,Urbana,Privada,10,10,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [23]:
len(indicador_media_horas_aula)

6952

In [26]:
indicador_media_horas_aula['categoria'].unique()

array(['Urbana', 'Rural'], dtype=object)

In [27]:
indicador_media_horas_aula['tipo'].unique()

array(['Estadual', 'Municipal', 'Privada', 'Federal'], dtype=object)

Renomeando colunas

In [24]:
indicador_media_horas_aula = indicador_media_horas_aula.rename(columns = {'CO_ENTIDADE':'id_escola', 'NO_ENTIDADE':'escola','NO_CATEGORIA':'categoria', 'NO_DEPENDENCIA':'tipo' }, inplace = False)

In [28]:
indicador_media_horas_aula = indicador_media_horas_aula.rename(columns = {'ED_INF_CAT_0':'total_ens_infantil', 'CRE_CAT_0':'creche_ens_infantil', 'PRE_CAT_0'	: 'pre_escola_ens_infantil' }, inplace = False)

In [29]:
indicador_media_horas_aula = indicador_media_horas_aula.rename(columns = {'FUN_CAT_0':'total_ens_fund', 'FUN_AI_CAT_0':'total_anos_inciais_ens_fund', 'FUN_AF_CAT_0': 'total_anos_finais_ens_fund','FUN_01_CAT_0':'ano1_ens_fund','FUN_02_CAT_0':'ano2_ens_fund', 'FUN_03_CAT_0':'ano3_ens_fund', 'FUN_04_CAT_0':'ano4_ens_fund', 'FUN_05_CAT_0':'ano5_ens_fund', 'FUN_06_CAT_0':'ano6_ens_fund', 'FUN_07_CAT_0':'ano7_ens_fund', 'FUN_08_CAT_0':'ano8_ens_fund', 'FUN_09_CAT_0':'ano9_ens_fund'}, inplace = False)

In [31]:
indicador_media_horas_aula = indicador_media_horas_aula.rename(columns = {'MULT_ETA_CAT_0':'turma_multi_estapas_ens_fund'}, inplace = False)

In [30]:
indicador_media_horas_aula = indicador_media_horas_aula.rename(columns = {'MED_CAT_0':'total_ens_medio','MED_01_CAT_0':'ano1_ens_med','MED_02_CAT_0':'ano2_ens_med','MED_03_CAT_0':'ano3_ens_med','MED_04_CAT_0':'ano4_ens_med','MED_NS_CAT_0':'nao_seriado_ens_medio' }, inplace = False)

In [32]:
indicador_media_horas_aula

,id_escola,escola,categoria,tipo,total_ens_infantil,creche_ens_infantil,pre_escola_ens_infantil,total_ens_fund,total_anos_inciais_ens_fund,total_anos_finais_ens_fund,ano1_ens_fund,ano2_ens_fund,ano3_ens_fund,ano4_ens_fund,ano5_ens_fund,ano6_ens_fund,ano7_ens_fund,ano8_ens_fund,ano9_ens_fund,total_ens_medio,ano1_ens_med,ano2_ens_med,ano3_ens_med,ano4_ens_med,nao_seriado_ens_medio
109290,35000012.0,AYRES DE MOURA PROFESSOR,Urbana,Estadual,--,--,--,8.7,--,8.7,--,--,--,--,--,8.5,8.8,8.8,8.8,--,--,--,--,--,--
109291,35000024.0,GAVIAO PEIXOTO BRIGADEIRO,Urbana,Estadual,--,--,--,5.4,5.3,5.5,4.9,5.3,5.3,5.3,5.3,5.3,5.3,5.6,5.6,5,5.3,4.8,4.7,--,--
109292,35000036.0,JOAO SOLIMEO,Urbana,Estadual,--,--,--,5.5,--,5.5,--,--,--,--,--,5.4,5.6,5.6,5.6,4.6,5.5,4,4,--,--
109293,35000048.0,WALFREDO ARANTES CALDAS PROFESSOR,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,5,5.2,4.9,4.5,--,--
109294,35000059.0,SEBASTIAO DE OLIVEIRA GUSMAO PROFESSOR,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,5,5.6,4.6,4.5,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116237,35925949.0,DOM AGNELO CARDEAL ROSSI,Urbana,Estadual,--,--,--,5.3,5.3,--,5.3,5.3,5.3,5.3,5.3,--,--,--,--,--,--,--,--,--,--
116238,35925998.0,ADELAIDE ROSA FERNANDES MACHADO DE SOUZA PROFE...,Urbana,Estadual,--,--,--,9.5,--,9.5,--,--,--,--,--,9.5,9.5,9.5,9.5,9.5,9.5,9.5,9.5,--,--
116239,35926048.0,BARRO BRANCO II,Urbana,Estadual,--,--,--,5,4.5,5.6,4.5,4.5,4.5,4.5,4.5,5.6,5.6,5.6,5.6,4,4,4,4,--,--
116240,35926103.0,PROFESSORA ELIANA PASSUELO,Urbana,Estadual,--,--,--,4.5,4.5,--,4.5,4.5,4.5,4.5,4.5,--,--,--,--,--,--,--,--,--,--


#**Tabela Dimensioanl: media_de_horas_aula_escolas_dim**

In [39]:
media_de_horas_aula_escolas_dim = pd.DataFrame()
media_de_horas_aula_escolas_dim['escola'] = indicador_media_horas_aula['escola']
media_de_horas_aula_escolas_dim['id_escola'] = indicador_media_horas_aula['id_escola']

In [40]:
media_de_horas_aula_escolas_dim

,escola,id_escola
109290,AYRES DE MOURA PROFESSOR,35000012.0
109291,GAVIAO PEIXOTO BRIGADEIRO,35000024.0
109292,JOAO SOLIMEO,35000036.0
109293,WALFREDO ARANTES CALDAS PROFESSOR,35000048.0
109294,SEBASTIAO DE OLIVEIRA GUSMAO PROFESSOR,35000059.0
...,...,...
116237,DOM AGNELO CARDEAL ROSSI,35925949.0
116238,ADELAIDE ROSA FERNANDES MACHADO DE SOUZA PROFE...,35925998.0
116239,BARRO BRANCO II,35926048.0
116240,PROFESSORA ELIANA PASSUELO,35926103.0


In [41]:
media_de_horas_aula_escolas_dim.to_csv(r'media_de_horas_aula_escolas_dim', index = False)

#**Concluindo a Fato: media_horas_aula_escolas_fato**

In [42]:
indicador_media_horas_aula.drop(columns =['escola'], inplace = True) 

In [43]:
indicador_media_horas_aula

,id_escola,categoria,tipo,total_ens_infantil,creche_ens_infantil,pre_escola_ens_infantil,total_ens_fund,total_anos_inciais_ens_fund,total_anos_finais_ens_fund,ano1_ens_fund,ano2_ens_fund,ano3_ens_fund,ano4_ens_fund,ano5_ens_fund,ano6_ens_fund,ano7_ens_fund,ano8_ens_fund,ano9_ens_fund,total_ens_medio,ano1_ens_med,ano2_ens_med,ano3_ens_med,ano4_ens_med,nao_seriado_ens_medio
109290,35000012.0,Urbana,Estadual,--,--,--,8.7,--,8.7,--,--,--,--,--,8.5,8.8,8.8,8.8,--,--,--,--,--,--
109291,35000024.0,Urbana,Estadual,--,--,--,5.4,5.3,5.5,4.9,5.3,5.3,5.3,5.3,5.3,5.3,5.6,5.6,5,5.3,4.8,4.7,--,--
109292,35000036.0,Urbana,Estadual,--,--,--,5.5,--,5.5,--,--,--,--,--,5.4,5.6,5.6,5.6,4.6,5.5,4,4,--,--
109293,35000048.0,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,5,5.2,4.9,4.5,--,--
109294,35000059.0,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,5,5.6,4.6,4.5,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116237,35925949.0,Urbana,Estadual,--,--,--,5.3,5.3,--,5.3,5.3,5.3,5.3,5.3,--,--,--,--,--,--,--,--,--,--
116238,35925998.0,Urbana,Estadual,--,--,--,9.5,--,9.5,--,--,--,--,--,9.5,9.5,9.5,9.5,9.5,9.5,9.5,9.5,--,--
116239,35926048.0,Urbana,Estadual,--,--,--,5,4.5,5.6,4.5,4.5,4.5,4.5,4.5,5.6,5.6,5.6,5.6,4,4,4,4,--,--
116240,35926103.0,Urbana,Estadual,--,--,--,4.5,4.5,--,4.5,4.5,4.5,4.5,4.5,--,--,--,--,--,--,--,--,--,--


In [44]:
indicador_media_horas_aula.to_csv(r'media_horas_aula_escolas_fato', index = False)